## 1) Setup + configuración

In [ ]:
from dotenv import load_dotenv
import os
from pathlib import Path


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "Missing OPENAI_API_KEY at .env"

DATA_DIR = Path("data")
assert DATA_DIR.exists(), "data/ folder does not exist!"

print("✅ Environment OK")

In [ ]:
BOE_PDF = DATA_DIR / "BOE-A-2020-8608.pdf"
SENTINEL_PDF = DATA_DIR / "sentinel_secure_services_maual_operativo.pdf"  # noqa

print("BOE exists:", BOE_PDF.exists())
print("Sentinel exists:", SENTINEL_PDF.exists())

## 2) Carga + extracción

In [ ]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader


def load_pdf(path: Path) -> list[Document]:
    loader = PyPDFLoader(str(path))
    documents = loader.load()
    return documents


docs_boe = load_pdf(BOE_PDF) if BOE_PDF.exists() else []
docs_sentinel = load_pdf(SENTINEL_PDF) if SENTINEL_PDF.exists() else []

docs = docs_boe
print(f"Pages loaded: {len(docs)}")
print(f"Example metadata: {docs[0].metadata}" if docs else None)
print(f"Sample text: {docs[0].page_content[:400]} [...]" if docs else "No docs")

In [ ]:
def doc_stats(documents: list[Document]) -> tuple[int, int]:
    total_characters = sum(len(d.page_content) for d in documents)
    average_characters = total_characters / max(1, len(documents))
    return total_characters, average_characters


total_chars, avg_chars = doc_stats(docs)
print(f"Total characters: {total_chars:_}\nAvg characters per page: {avg_chars:_.0f}")

## 3) Chunking

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


def split_docs(documents: list[Document], chunk_size: int = 800, chunk_overlap: int = 120) -> list[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],
    )
    return splitter.split_documents(documents)


CHUNK_SIZE = 100
CHUNK_OVERLAP = 10

docs_chunks = split_docs(docs, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

print(f"{len(docs_chunks)} chunks (size {CHUNK_SIZE}, overlap {CHUNK_OVERLAP})")

In [ ]:
import textwrap


def preview_chunks(chunks: list[Document], n: int = 2, width: int = 100, max_chars: int = 800) -> None:
    for i in range(min(n, len(chunks))):
        md = chunks[i].metadata
        txt = chunks[i].page_content.strip().replace("\n", " ")
        print(f"\n—Chunk {i} | source={md.get('source')}; page={md.get('page')}; len={len(chunks[i].page_content):_}")
        print(textwrap.fill(txt[:max_chars], width=width))


preview_chunks(docs_chunks, n=2, max_chars=800)

## 4) Embeddings + vector store

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


EMBED_MODEL = "text-embedding-3-small"


def build_chroma(chunks: list[Document], persist_dir="chroma_boe", collection="boe") -> Chroma:
    embeddings = OpenAIEmbeddings(model=EMBED_MODEL)
    db = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_dir,
        collection_name=collection,
    )
    return db


db_chunks = build_chroma(docs_chunks, persist_dir=f"chroma_{CHUNK_SIZE}", collection=f"size_{CHUNK_SIZE}")
print(f"✅ Chroma built (size_{CHUNK_SIZE}).")

## 5) Retrieval

In [ ]:
def retrieve_with_scores(db: Chroma, query_: str, n_results: int = 4) -> list[tuple[Document, float]]:
    return db.similarity_search_with_score(query_, k=n_results)


def print_chroma_hits(hits: list[tuple[Document, float]], max_chars: int = 600, width: int = 110) -> None:
    print(f"Score: less is better for cosine-distance based search.")
    for i, (doc, score) in enumerate(hits):
        md = doc.metadata
        print(f"\n#{i} score={score:.4f}; page={md.get('page')}; source={md.get('source')}")
        print(textwrap.fill(doc.page_content[:max_chars].replace("\n", " "), width=width))


query = "¿Cuál es el porcentaje máximo de subvención para proyectos en fase comercial?"  # noqa
results = retrieve_with_scores(db_chunks, query, n_results=5)
print_chroma_hits(results, max_chars=600)

## Experimento 1

El pipeline funciona tal como esperamos, pero los resultados no parecen buenos... ¿por qué?

Prueba con 3 o 4 combinaciones de CHUNK_SIZE y CHUNK_OVERLAP. Analiza los fragmentos generados, los top-5 fragmentos recuperados.

¿Qué impacto crees que tendrá esta decisión en las respuestas finales del sistema?

## 6) Generación

In [ ]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)


def answer_no_context(question_: str) -> str:
    return llm.invoke(question_).content


def answer_with_context(question_: str, retrieved_docs: list[Document]) -> str:
    context = "\n\n".join(
        f"[source={d.metadata.get('source')}; page={d.metadata.get('page')}]\n{d.page_content}"
        for d in retrieved_docs
    )

    # noinspection SpellCheckingInspection
    prompt = f"""Contexto: {context}\nPregunta: {question_}"""
    return llm.invoke(prompt).content

In [ ]:
question = "¿Cuál es el porcentaje máximo de subvención para proyectos en fase comercial?"  # noqa

print("=== No context ===")
print(answer_no_context(question))

print(f"\n=== With RAG context (chunk size {CHUNK_SIZE}) ===")
retrieved = [doc for doc, _score in retrieve_with_scores(db_chunks, question, n_results=1)]
print(answer_with_context(question, retrieved))

## Experimento 2

Acabamos de ver que elegir un buen CHUNK_SIZE es importante. Sin embargo, en la práctica intentamos tener sistemas más robustos incluso para parámetros subóptimos.

Prueba a cambiar el valor de n_results para distintos valores de CHUNK_SIZE y observa los resultados. ¿Crees que existe una correlación entre CHUNK_SIZE y n_results?

## Experimento 3

Hemos probado a trabajar con un prompt muy básico:

```prompt = f"""Contexto: {context}\nPregunta: {question_}"""```

Prueba a lanzar una pregunta claramente fuera del contexto del documento de nuestra base documental: ¿qué ocurre?

¿Puedes conseguir que solo responda cuando la respuesta se encuentre en el contexto? (¿Es este el comportamiento idóneo?)

¿Puedes conseguir que la respuesta sea más verbosa? ¿Y más esquemática pero citando detalles de la fuente utilizada?